# Application for Reading & Updating XML with NER

In [1]:
import warnings, re, glob, datetime, csv, sys, os, base64, io, spacy
import pandas as pd
import numpy as np
from lxml import etree

import dash, dash_table
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import dash_html_components as html
from jupyter_dash import JupyterDash

# Import spaCy language model.
nlp = spacy.load('en_core_web_sm')

# Ignore simple warnings.
warnings.simplefilter('ignore', DeprecationWarning)

# Declare directory location to shorten filepaths later.
abs_dir = "/Users/quinn.wi/Documents/GitHub/dsg-mhs/"

## Declare Functions

In [2]:
%%time

"""
XML Parsing Function: Get Namespaces
"""
def get_namespace(root):
    namespace = re.match(r"{(.*)}", str(root.tag))
    ns = {"ns":namespace.group(1)}
    return ns


"""
XML Parsing Function: Retrieve XPaths
"""
def get_abridged_xpath(child):
    if child.getparent().get('{http://www.w3.org/XML/1998/namespace}id') is not None:    
        ancestor = child.getparent().tag
        xml_id = child.getparent().get('{http://www.w3.org/XML/1998/namespace}id')

        abridged_xpath = f'.//ns:body//{ancestor}[@xml:id="{xml_id}"]/{child.tag}'
        return abridged_xpath


"""
XML Parsing Function: Convert to String
"""
def get_text(elem):
    text_list = []
    text = ''.join(etree.tostring(elem, encoding='unicode', method='text', with_tail=False))
    text_list.append(re.sub(r'\s+', ' ', text))
    return ' '.join(text_list)

        
"""
XML Parsing Function: Get Encoded Content
"""    
def get_encoding(elem):
    encoding = etree.tostring(elem, pretty_print = True).decode('utf-8')
    encoding = re.sub('\s+', ' ', encoding) # remove additional whitespace
    return encoding


  

"""
NER Function
"""
# spaCy
def get_spacy_entities(text, subset_ner):
    sp_entities_l = []
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in subset_ner.keys():
            sp_entities_l.append((str(ent), ent.label_))
        else:
            pass
    return sp_entities_l




"""
XML & NER: Retrieve Contents
"""
def get_contents(ancestor, xpath_as_string, namespace, subset_ner):
    
    textContent = get_text(ancestor) # Get plain text.
    encodedContent = get_encoding(ancestor) # Get encoded content.
    sp_entities_l = get_spacy_entities(textContent, subset_ner) # Get named entities from plain text.

    return (sp_entities_l, encodedContent)




"""
XML Parsing Function: Write New Encoding with Up-Conversion
"""
def make_ner_suggestions(previous_encoding, entity, label, subset_ner, kwic_range, banned_list):
#     Regularize spacing & store data as new variable ('converted_encoding').
    converted_encoding = re.sub('\s+', ' ', previous_encoding, re.MULTILINE)
    
#     Create regex that replaces spaces with underscores if spaces occur within tags.
#     This regex treats tags as a single token later.
    tag_regex = re.compile('<(.*?)>')

#     Accumulate underscores through iteration
    for match in re.findall(tag_regex, previous_encoding):
        replace_space = re.sub('\s', '_', match)
        converted_encoding = re.sub(match, replace_space, converted_encoding)
    
#     Up-convert entity (label remains unchanged).
    label = subset_ner[label]    
    converted_entity = ' '.join(['<w>' + e + '</w>' for e in entity.split(' ')])
    
#     Up-Converstion
#     Tokenize encoding and text, appending <w> tags, and re-join.
    converted_encoding = converted_encoding.split(' ')
    for idx, item in enumerate(converted_encoding):
        item = '<w>' + item + '</w>'
        converted_encoding[idx] = item
        
    converted_encoding = ' '.join(converted_encoding)
    
#     Find converted entities and kwic-converted entities, even if there's additional encoding within entity.
    try:
        entity_regex = re.sub('<w>(.*)</w>', '(\\1)(.*?</w>)', converted_entity)
        entity_match = re.search(entity_regex, converted_encoding)
        
        ban_decision = []
        for i in banned_list:
            if i in entity_match.group(0):
                ban_decision.append('y')
                
        if 'y' in ban_decision:
            return "Already Encoded"
        
#         If expanded regex is in previous encoding, find & replace it with new encoding.
        elif entity_match:
            new_encoding = re.sub(f'{entity_match.group(0)}',
                                  f'<{label}>{entity_match.group(1)}</{label}>{entity_match.group(2)}',
                                  converted_encoding)
            
#             Remove <w> tags to return to well-formed xml.
            new_encoding = re.sub('<[/]?w>', '', new_encoding)
#             Remove underscores.
            new_encoding = re.sub('_', ' ', new_encoding)

            return new_encoding

        else:
            return 'Error Making NER Suggestions'
    
#     Up-conversion works well because it 'breaks' if an entity already has been encoded:
#     <w>Abel</w> (found entity) does not match <w><persRef_ref="abel-mary">Mrs</w> <w>Abel</persRef></w>
#     <persRef> breaks function and avoids duplicating entities.
    
    except:
        return 'Error Occurred with Regex.'
        



"""
XML: & NER: Create Dataframe of Entities
"""
def make_dataframe(child, df, ns, subset_ner, filename, descendant_order):
    abridged_xpath = get_abridged_xpath(child)
    entities, previous_encoding = get_contents(child, './/ns:.', ns, subset_ner)

    df = df.append({
        'file':re.sub('.*/(.*.xml)', '\\1', filename),
        'descendant_order': descendant_order,
#         'abridged_xpath':abridged_xpath,
        'previous_encoding': previous_encoding,
        'entities':entities,
    },
        ignore_index = True)
    
    return df



"""
Parse Contents: XML Structure (ouput-data-upload)
"""
def parse_contents(contents, filename, date, ner_values):
    ner_values = ner_values.split(',')
    content_type, content_string = contents.split(',')
    decoded = base64.b64decode(content_string).decode('utf-8')
    
    # Label dictionary.
    label_dict = {'PERSON':'persName',
                  'LOC':'placeName', # Non-GPE locations, mountain ranges, bodies of water.
                  'GPE':'placeName', # Countries, cities, states.
                  'FAC':'placeName', # Buildings, airports, highways, bridges, etc.
                  'ORG':'orgName', # Companies, agencies, institutions, etc.
                  'NORP':'name', # Nationalities or religious or political groups.
                  'EVENT':'name', # Named hurricanes, battles, wars, sports events, etc.
                  'WORK_OF_ART':'name', # Titles of books, songs, etc.
                  'LAW':'name', # Named documents made into laws.
                  'DATE':'date' # Absolute or relative dates or periods.
                 }
    
    #### Subset label_dict with input values from Checklist *****
    subset_ner = {k: label_dict[k] for k in ner_values}
    
#     Run XML Parser + NER here.
    try:
#         Assume that the user uploaded a CSV file
        if 'csv' in filename:
            df = pd.read_csv(
                io.StringIO(decoded)
            )
            
#         Assume that the user uploaded an XML file
        elif 'xml' in filename:
            xml_file = decoded.encode('utf-8')
            
            df = pd.DataFrame(columns = ['file', 'abridged_xpath', 'previous_encoding', 'entities'])
            
            root = etree.fromstring(xml_file)
            ns = get_namespace(root)
            
#             Search through elements for entities.
            desc_order = 0
            for child in root.findall('.//ns:body//ns:div[@type="docbody"]', ns):
            
                abridged_xpath = get_abridged_xpath(child)
                
                for descendant in child:
                    desc_order = desc_order + 1
                    df = make_dataframe(descendant, df, ns, subset_ner, filename, desc_order)
                    df['abridged_xpath'] = abridged_xpath
                
#             Join data
            df = df \
                .explode('entities') \
                .dropna()

            df[['entity', 'label']] = pd.DataFrame(df['entities'].tolist(), index = df.index)
            
            df['new_encoding'] = df \
                .apply(lambda row: make_ner_suggestions(row['previous_encoding'],
                                                        row['entity'],
                                                        row['label'],
                                                        subset_ner, 4, banned_list),
                       axis = 1)

            
            # Add additional columns for user input.
            df['accept_changes'] = ''
            df['make_hand_edits'] = ''
            
#             Drop rows if 'new_encoding' value equals 'Already Encoded'.
            df = df[df['new_encoding'] != 'Already Encoded']

            
    except Exception as e:
        return html.Div([
            f'There was an error processing this file: {e}.'
    ])


#     Return HTML with outputs.
    return filename, date, df



"""
XML & Regex: Up Conversion

Function replaces all spaces between beginning and end tags with underscores.
Then, function wraps each token (determined by whitespace) with word tags (<w>...</w>)
"""
def up_convert_encoding(column):
#     Regularize spacing & store data as new variable ('converted_encoding').
    converted_encoding = re.sub('\s+', ' ', column, re.MULTILINE)
    
#     Create regex that replaces spaces with underscores if spaces occur within tags.
#     This regex treats tags as a single token later.
    tag_regex = re.compile('<(.*?)>')

#     Accumulate underscores through iteration
    for match in re.findall(tag_regex, column):
        replace_space = re.sub('\s', '_', match)
        converted_encoding = re.sub(match, replace_space, converted_encoding)
    
#     Up-Converstion
#     Tokenize encoding and text, appending <w> tags, and re-join.
    converted_encoding = converted_encoding.split(' ')
    for idx, item in enumerate(converted_encoding):
        item = '<w>' + item + '</w>'
        converted_encoding[idx] = item
    converted_encoding = ' '.join(converted_encoding)
    
    return converted_encoding




"""
XML Parsing Function: Suggest New Encoding with Hand Edits

Similar to make_ner_suggestions(), this function folds in revision using regular expressions.
The outcome is the previous encoding with additional encoded information determined by user input.

Expected Columns:
    previous_encoding
    entities
    accept_changes
    make_hand_edits
"""
def revise_with_hand_edits(label_dict, make_hand_edits, 
                           label, entity, previous_encoding, new_encoding):
    
    label = label_dict[label]
    
#     Up convert PREVIOUS ENCODING: assumes encoder will supply new encoding and attribute with value.
    converted_encoding = up_convert_encoding(previous_encoding)
    converted_entity = ' '.join(['<w>' + e + '</w>' for e in entity.split(' ')])

#     If there is a unique id to add & hand edits...
    if make_hand_edits != '':
        
        entity_regex = re.sub('<w>(.*)</w>', '(\\1)(.*?</w>)', converted_entity)
        entity_match = re.search(entity_regex, converted_encoding)

        identifier_regex = re.search('<(.+)>(.+)</.+>', make_hand_edits, re.VERBOSE|re.DOTALL)

        revised_encoding = re.sub(f'{entity_match.group(0)}',
                                          f'<{label}>{entity_match.group(1)}</{label}>{entity_match.group(2)}',
                                          converted_encoding)


        revised_encoding = re.sub(f'<{label}>', f'<{identifier_regex.group(1)}>', revised_encoding)

#         Clean up any additional whitespace and remove word tags.
        revised_encoding = re.sub('\s+', ' ', revised_encoding, re.MULTILINE)
        revised_encoding = re.sub('<[/]?w>', '', revised_encoding)
        
        revised_encoding = re.sub('_', ' ', revised_encoding) # Remove any remaining underscores in tags.
        revised_encoding = re.sub('“', '"', revised_encoding) # Change quotation marks to correct unicode.
        revised_encoding = re.sub('”', '"', revised_encoding)
        
        return revised_encoding

    else:
        pass



"""
XML & NER: Update/Inherit Accepted Changes
Expects a dataframe (from a .csv) with these columns:
    file
    abridged_xpath
    descendant_order
    previous_encoding
    entities
    new_encoding
    accept_changes
    make_hand_edits
"""
def inherit_changes(label_dict, dataframe):
    
    dataframe = dataframe.fillna('')
    for index, row in dataframe.iterrows():
        
#         If HAND changes are accepted...
        if row['make_hand_edits'] != '':
        
            revised_by_hand = revise_with_hand_edits(label_dict, 
                                                     row['make_hand_edits'],
                                                     row['label'], row['entity'], 
                                                     row['previous_encoding'], row['new_encoding'])

            dataframe.loc[index, 'new_encoding'] = revised_by_hand
            
            try:
                if dataframe.loc[index + 1, 'abridged_xpath'] == row['abridged_xpath'] \
                and dataframe.loc[index + 1, 'descendant_order'] == row['descendant_order']:
                    dataframe.loc[index + 1, 'previous_encoding'] = row['new_encoding']
                    
                else:
                    dataframe.loc[index, 'new_encoding'] = revised_by_hand
                    
                    
            except KeyError as e:
                dataframe.loc[index, 'new_encoding'] = revised_by_hand
        
#         If NER suggestions are accepted as-is...
        elif row['accept_changes'] == 'y' and row['make_hand_edits'] == '':
        
            try:
                if dataframe.loc[index + 1, 'abridged_xpath'] == row['abridged_xpath'] \
                and dataframe.loc[index + 1, 'descendant_order'] == row['descendant_order']:
                    dataframe.loc[index + 1, 'previous_encoding'] = row['new_encoding']
                
                else:
                    dataframe.loc[index, 'new_encoding'] = row['new_encoding']
                    
            except KeyError as e:
                dataframe.loc[index, 'new_encoding'] = row['new_encoding']
                
#         If changes are rejected...
        else:
            try:
                if dataframe.loc[index + 1, 'abridged_xpath'] == row['abridged_xpath'] \
                and dataframe.loc[index + 1, 'descendant_order'] == row['descendant_order']:
                    dataframe.loc[index + 1, 'previous_encoding'] = dataframe.loc[index, 'previous_encoding']
                    
            except KeyError as e:
                dataframe.loc[index, 'new_encoding'] = dataframe.loc[index, 'previous_encoding']

#     Subset dataframe with finalized revisions.
    dataframe = dataframe.groupby(['abridged_xpath', 'descendant_order']).tail(1)
    
    return dataframe


"""
XML & NER: Write New XML File with Accepted Revisions
Expects:
    XML File with Original Encoding
    CSV File with Accepted Changes
    Label Dictionary
"""
def revise_xml(xml_contents, csv_df):
#     Label dictionary.
    label_dict = {'PERSON':'persName',
                  'LOC':'placeName', # Non-GPE locations, mountain ranges, bodies of water.
                  'GPE':'placeName', # Countries, cities, states.
                  'FAC':'placeName', # Buildings, airports, highways, bridges, etc.
                  'ORG':'orgName', # Companies, agencies, institutions, etc.
                  'NORP':'name', # Nationalities or religious or political groups.
                  'EVENT':'name', # Named hurricanes, battles, wars, sports events, etc.
                  'WORK_OF_ART':'name', # Titles of books, songs, etc.
                  'LAW':'name', # Named documents made into laws.
                  'DATE':'date' # Absolute or relative dates or periods.
                 }
    
#     First, update data to reflect accepted changes.
    new_data = inherit_changes(label_dict, csv_df)
    
    xml_content_type, xml_content_string = xml_contents.split(',')
    xml_decoded = base64.b64decode(xml_content_string).decode('utf-8')
    xml_file = xml_decoded.encode('utf-8')
    
    root = etree.fromstring(xml_file)
    ns = get_namespace(root)
    
    tree_as_string = etree.tostring(root, pretty_print = True).decode('utf-8')
    tree_as_string = re.sub('\s+', ' ', tree_as_string) # remove additional whitespace
    
#     Write accepted code into XML tree.
    for index, row in new_data.iterrows():
        original_encoding_as_string = row['previous_encoding']
        
        # Remove namespaces within tags to ensure regex matches accurately.
        original_encoding_as_string = re.sub('^<(.*?)( xmlns.*?)>(.*)$',
                                             '<\\1>\\3',
                                             original_encoding_as_string)
        
        accepted_encoding_as_string = row['new_encoding']
        accepted_encoding_as_string = re.sub('<(.*?)( xmlns.*?)>(.*)$',
                                             '<\\1>\\3',
                                             accepted_encoding_as_string) # Remove namespaces within tags.
        
        tree_as_string = re.sub(original_encoding_as_string,
                                accepted_encoding_as_string,
                                tree_as_string)

        
#     Check well-formedness (will fail if not well-formed)
    doc = etree.fromstring(tree_as_string)
    et = etree.ElementTree(doc)
    
#     Convert to string.
    et = etree.tostring(et, encoding='unicode', method='xml', pretty_print = True)
    return et

CPU times: user 6 µs, sys: 1 µs, total: 7 µs
Wall time: 9.3 µs


## APP

In [3]:
%%time

# External JavaScript files
external_scripts = [
    'https://ajax.googleapis.com/ajax/libs/jquery/2.1.1/jquery.min.js',
    {'src':'https://ajax.googleapis.com/ajax/libs/jquery/2.1.1/jquery.min.js'}
]


app = JupyterDash(__name__, 
                  external_scripts=external_scripts,)
app.config.suppress_callback_exceptions = True


# Preset variables.
ner_labels = ['PERSON','LOC','GPE','FAC','ORG','NORP','EVENT','WORK_OF_ART','LAW','DATE']

# Banned List (list of elements that already encode entities)
banned_list = ['persRef', 'persName', 'date']

# Layout.
app.layout = html.Div([
    
#     Title
    html.Div(
        className="app-header",
        children = [
            html.Div('NER + XML Reader', className = "app-header--title")
        ]),

#     Add or substract labels to list for NER to find. Complete list of NER labels: https://spacy.io/api/annotation
    html.H2('Select Entities to Search For'),
    dcc.Checklist(
        id = 'ner-checklist',
        options = [{
            'label': i,
            'value': i
        } for i in ner_labels],
        value = ['PERSON', 'LOC', 'GPE']
    ),
    
    
#     Upload Data Area.
    html.H2('Upload File'),
    dcc.Upload(
        id = 'upload-data',
        children = html.Div([
            'Drag and Drop or ', html.A('Select File')
        ]),
        style={
            'width': '95%',
            'height': '60px',
            'lineHeight': '60px',
            'borderWidth': '1px',
            'borderStyle': 'dashed',
            'borderRadius': '5px',
            'textAlign': 'center',
            'margin': '10px'
        },
        multiple=True # Allow multiple files to be uploaded
    ),
    
#     Store uploaded data.
    dcc.Store(id = 'data-upload-store'),
    
#     Display pane for file information.
    html.Div(id = 'file-information'),
    
#     Display pane for data as table.
    dash_table.DataTable(id = 'data-table-container',
                         row_selectable="single",
                         selected_rows = [0],
                         editable = True,
                         page_size=1,
                        ),
    
#     Display pane for reading data from selected row & revision options.
    html.Div(id = 'reading-container'),
    html.Div(id = 'revision-radio-container', style = {'display': 'inline-block'}),
    html.Div(id = 'revision-text-container', style = {'display': 'inline-block'}),
    html.Div(id = 'revision-button-container'),
    
#     Store revised data.
    dcc.Store(id = 'revisions-store'),
    
#     Div to hold button that will write and download XML file.
    html.Div(id = 'write-button-container'),
    
    html.Div(id = 'download-button-container')

])


# Upload data & create table.
@app.callback([Output('file-information', 'children'),
               Output('data-upload-store', 'data')],
              [Input('upload-data', 'contents'),
               Input('ner-checklist', 'value')],
              [State('upload-data', 'filename'),
               State('upload-data', 'last_modified')])
def upload_data(list_of_contents, ner_values, list_of_names, list_of_dates):
    if list_of_contents is None:
        raise PreventUpdate
    
#     Parse uploaded contents.
    children = [
        parse_contents(c, n, d, ner) for c, n, d, ner in
        zip(list_of_contents, list_of_names, list_of_dates, ner_values)
    ]

    filename = children[0][0]
    date = children[0][1]
    data = children[0][2]
    
#     Extract file information.
    file_information = html.Div([html.P(f'File name: {filename}'),
                                 html.P(f'Last Modified: {datetime.datetime.fromtimestamp(date)}')])    
    
    return file_information, data.to_dict('rows')



# Generate table with data from store.
@app.callback([Output('data-table-container', 'data'),
               Output('data-table-container', 'columns')],
              Input('data-upload-store', 'data'))
def populate_data_table(data):

    df = pd.DataFrame(data)[['file', 'entity', 'label']]
    cols = [{'name':i, 'id': i} for i in df.columns]

    return df.to_dict('rows'), cols



# Create reading pane & revision options once row from table is selected.
@app.callback([Output('reading-container', 'children'),
               Output('revision-radio-container', 'children'),
               Output('revision-text-container', 'children'),
               Output('revision-button-container', 'children')],
              [Input('data-upload-store', 'data'),
               Input('data-table-container', 'selected_rows')])
def create_reading_and_revisions_pane(data, selected_rows):
    if data is None:
        raise PreventUpdate
        
    reading_df = pd.DataFrame(data).iloc[selected_rows]
    
#     Access previous and new encoding and convert to the "code" html element.
    prev_encoding = html.Code(reading_df['previous_encoding'])
    new_encoding = html.Code(reading_df['new_encoding'])
    
    reading_pane = html.Div([
        html.H2('Suggested Revisions'),
#         Create html table.
        html.Table([
#             Create headers.
            html.Tr([
                html.Th('Previous Encoding'),
                html.Th('New Encoding')
            ]),
#             Create row values.
            html.Tr([
                html.Td(prev_encoding),
                html.Td(new_encoding)
            ])
        ]),
        
        html.H2('Revisions Options'),
    ])
    
#     reading_pane = html.Div([
#         html.P(reading_df['previous_encoding'], id = 'old'),
#         html.P(reading_df['new_encoding'], id = 'new'),
        
#         html.H2('Revisions Options'),
#     ])
    
#     Create accept (as-is) or reject buttons.
    revision_radio = dcc.RadioItems(
            id = 'radioInput',
            options = [
                {'label':'Accept Changes As-Is', 'value':'y'},
                {'label':'Reject Changes', 'value':''}
            ],
#             labelStyle = {'display':'inline-block'}
        )
        
#     Create text area for manual changes.
    revision_text = dcc.Input(id = 'textInput', type = 'text', 
                              placeholder = 'Hand type changes here', value = '', debounce = True)
        
#     Create button for committing changes.
    revision_button = html.Button('Confirm Changes?', id = 'confirm-button', n_clicks = 0)
    
    return reading_pane, revision_radio, revision_text, revision_button
    

# Once a revisions is accepted, write row with instructions to dataframe.
@app.callback(Output('revisions-store', 'data'),
              [Input('revision-button-container', 'n_clicks'),
               Input('data-upload-store', 'data'),
               Input('data-table-container', 'selected_rows'),
               Input('revision-radio-container', 'children'), 
               Input('revision-text-container', 'children')],
              State('revisions-store', 'data'))
def commit_revisions_to_dataframe(n_clicks, data, selected_rows,
                                  radio_children, text_children, revisions):
    
#     Only run if the n_click 'id' is triggered by the revision-button-container.
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
        
    if changed_id != 'revision-button-container.n_clicks':
        raise PreventUpdate
    
    revised_row = pd.DataFrame(data).iloc[selected_rows]
        
#     Check if radio_ and text_children each have a value by seeing if a 'value' key is nested in 'props'.
    if 'value' in radio_children['props']:
        radio_value = radio_children['props']['value']
    else:
        radio_value = ''
    
    if 'value' in text_children['props']:
        text_value = text_children['props']['value']
    else:
        text_value = ''
    
#     Change individual cell value according to selected row & user-input.
    revised_row['make_hand_edits'] = text_value
    revised_row['accept_changes'] = radio_value

#     Create or update revisions dataframe to store revisions.
    if revisions is None:
        revisions = pd.DataFrame(revised_row)
    else:
        revisions = pd.DataFrame(revisions)
        revisions = revisions.append(revised_row, ignore_index = True)
    
    return revisions.to_dict('rows')


# After last revision (or whenever one change completed), provide button to commit changes to XML.
@app.callback(Output('write-button-container', 'children'),
              Input('revisions-store', 'data'))
def provide_button_to_download_revisions(data):
    if data is None:
        raise PreventUpdate
    
    return html.Button('Finished? Download Revised XML.', id = 'write-xml-button')


# Run functions to revise XML and download new document.
@app.callback(Output('download-button-container', 'children'),
              [Input('write-button-container', 'n_clicks'),
               Input('upload-data', 'contents'),
               Input('revisions-store', 'data')],
              State('upload-data', 'filename'))
def provide_download_link(n_clicks, contents, revisions, filename):
    write_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
        
    if write_id != 'write-button-container.n_clicks':
        raise PreventUpdate
    
    xml_contents = contents[0]
    revisions = pd.DataFrame(revisions)
    
    final_revisions = revise_xml(xml_contents, revisions)
    
    path = f"revised-{filename[0]}"
    with open(path, "w") as file:
        file.write(final_revisions)

    return f'{filename[0]} downloaded!'

if __name__ == "__main__":
#     app.run_server(mode = 'inline', debug = True) # mode = 'inline' for JupyterDash
    app.run_server(debug = True)

Dash app running on http://127.0.0.1:8050/
CPU times: user 25.1 ms, sys: 10 ms, total: 35.2 ms
Wall time: 50.7 ms


In [4]:
# https://stackoverflow.com/questions/38037163/how-to-highlight-the-difference-of-two-texts-with-css

# https://webdevtrick.com/javascript-highlight-text/

# https://github.com/matthijsgroen/html-diff-js